**Common commands**

In [ ]:
# Save the model
model.save('path_to_your_model')

# Load the model
loaded_model = tf.keras.models.load_model('path_to_your_model')

In [ ]:
#To delete a directory
import shutil

# Define the directory to delete
directory_to_delete = 'PATH'

# Use shutil.rmtree() to delete the directory
shutil.rmtree(directory_to_delete)

print("Directory and its contents deleted successfully.")


Directory and its contents deleted successfully.


In [ ]:
#to move a directory
import shutil

source_dir = '/path/to/source_directory'
destination_dir = '/path/to/destination_directory'

# Move the directory
shutil.move(source_dir, destination_dir)

print("Directory moved successfully.")


In [ ]:
#To know how many files are in the directory
import os

directory = 'Path'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)


In [ ]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
source_dir = 'path'
destination_dir = 'path'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Define parameters
img_height = 224
img_width = 224
batch_size = 32

# Create ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=50,
    fill_mode='nearest'
)

# Load images from the source directory
image_files = os.listdir(source_dir)

# Augment and save the images
for filename in image_files:
    try:
        img = cv2.imread(os.path.join(source_dir, filename))
        if img is None:
            print(f"Failed to load image: {filename}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB format

        # Generate augmented images
        i = 0
        img = img.reshape((1,) + img.shape)  # Reshape image for the flow method
        for batch in datagen.flow(img, batch_size=batch_size, save_to_dir=destination_dir, save_prefix='aug', save_format='jpg'):
            i += 1
            if i >= 30:  # Generate 30 augmented images per original image
                break  # Exit the loop to move to the next image
    except Exception as e:
        print(f"Error processing image {filename}: {e}")

print("Augmentation completed.")


**PIPE DETECTION**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import random
import shutil

# Define the directories
dataset_dir = '/path/to/labeled_dataset'
train_dir = '/path/to/training_set'
val_dir = '/path/to/validation_set'
test_dir = '/path/to/testing_set'

# Define the labels (folder names)
labels = ['pipes', 'not_pipe']

# Define the ratios for splitting
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)
    for label in labels:
        os.makedirs(os.path.join(directory, label), exist_ok=True)

# Iterate through each label directory and split the images
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    images = os.listdir(label_dir)
    random.shuffle(images)

    num_images = len(images)
    train_split = int(train_ratio * num_images)
    val_split = int((train_ratio + val_ratio) * num_images)

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    for img in train_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(train_dir, label, img)
        shutil.move(src, dst)

    for img in val_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(val_dir, label, img)
        shutil.move(src, dst)

    for img in test_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(test_dir, label, img)
        shutil.move(src, dst)

print("Dataset split completed.")


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = '/content/drive/MyDrive/dataset/pipes_dataset/training'
val_dir = '/content/drive/MyDrive/dataset/pipes_dataset/validation'

# Define parameters
batch_size = 32
img_height = 224
img_width = 224

# Create ImageDataGenerator for training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create ImageDataGenerator for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Get the list of classes in the training directory
classes = os.listdir(train_dir)

# Load and label training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and label validation data, excluding .ipynb_checkpoints
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)


Found 919 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
import os

directory = '/content/drive/MyDrive/dataset/pipes_dataset/training'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)


Number of files in the directory: 919


In [ ]:
import os

directory = '/content/drive/MyDrive/dataset/pipes_dataset/validation'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)


Number of files in the directory: 306


In [ ]:
import os

directory = '/content/drive/MyDrive/dataset/pipes_dataset/testing'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)


Number of files in the directory: 307


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Initialize the model
model = Sequential()

# Add convolutional layers with increasing depth
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so use sigmoid activation

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Augment training data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/pipes_dataset/training', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')

# Load validation data
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/dataset/pipes_dataset/validation', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')

# Train the model
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=20, validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/pipes_dataset/testing', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')
eval_result = model.evaluate(test_generator)

print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])


Found 919 images belonging to 2 classes.
Found 306 images belonging to 2 classes.
Epoch 1/20
28/28 [==============================] - 226s 8s/step - loss: 0.8255 - accuracy: 0.5648 - val_loss: 0.6038 - val_accuracy: 0.6736
Epoch 2/20
28/28 [==============================] - 68s 2s/step - loss: 0.6020 - accuracy: 0.6697 - val_loss: 0.5592 - val_accuracy: 0.6875
Epoch 3/20
28/28 [==============================] - 73s 3s/step - loss: 0.5294 - accuracy: 0.7204 - val_loss: 0.5253 - val_accuracy: 0.7153
Epoch 4/20
28/28 [==============================] - 69s 2s/step - loss: 0.5098 - accuracy: 0.7463 - val_loss: 0.5124 - val_accuracy: 0.7326
Epoch 5/20
28/28 [==============================] - 71s 3s/step - loss: 0.5047 - accuracy: 0.7610 - val_loss: 0.4752 - val_accuracy: 0.7743
Epoch 6/20
28/28 [==============================] - 73s 3s/step - loss: 0.4589 - accuracy: 0.7779 - val_loss: 0.4425 - val_accuracy: 0.7986
Epoch 7/20
28/28 [==============================] - 79s 3s/step - loss: 0.423

In [ ]:
# Save the model
model.save('/content/drive/MyDrive/dataset/pipe_model/model1.h5')


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('/content/drive/MyDrive/dataset/pipe_model/model1.h5')  # Replace 'your_model.h5' with the path to your trained model

# Function to preprocess input image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_image(image_path):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    return predictions

# Function to interpret predictions
def interpret_predictions(predictions):
    class_index = int(predictions[0][0])  # Assuming predictions is of shape (1, 1)
    confidence = predictions[0][0] if class_index == 1 else 1 - predictions[0][0]
    if class_index == 0:
        return "No pipes are detected in the image", confidence
    elif class_index == 1:
        return "Pipe is detected in the image", confidence

# Example usage
image_path = '/content/drive/MyDrive/dataset/Figura-1-Ducto-submarino-con-claro-libre-no-soportado-ADSUN-2008vvvvvvvvv.png'  # Replace 'path_to_your_image.jpg' with the path to your image
predictions = predict_image(image_path)
interpretation, confidence = interpret_predictions(predictions)
print(interpretation)
print(f"Confidence: {confidence}")


1/1 [==============================] - 0s 118ms/step
Pipe is detected in the image
Confidence: 1.0


**RUST DETECTION**

In [ ]:
#To know how many files are in the directory
import os

directory = '/content/drive/MyDrive/dataset/rust_dataset/data/no_rust'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)

Number of files in the directory: 1534


In [ ]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
source_dir = '/content/drive/MyDrive/dataset/rust_dataset/rusted'
destination_dir = '/content/drive/MyDrive/dataset/rust_dataset/data/rust'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Define parameters
img_height = 224
img_width = 224
batch_size = 32

# Create ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=50,
    fill_mode='nearest'
)

# Load images from the source directory
image_files = os.listdir(source_dir)

# Augment and save the images
for filename in image_files:
    try:
        img = cv2.imread(os.path.join(source_dir, filename))
        if img is None:
            print(f"Failed to load image: {filename}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB format

        # Generate augmented images
        i = 0
        img = img.reshape((1,) + img.shape)  # Reshape image for the flow method
        for batch in datagen.flow(img, batch_size=batch_size, save_to_dir=destination_dir, save_prefix='aug', save_format='jpg'):
            i += 1
            if i >= 30:  # Generate 30 augmented images per original image
                break  # Exit the loop to move to the next image
    except Exception as e:
        print(f"Error processing image {filename}: {e}")

print("Augmentation completed.")


Augmentation completed.


In [ ]:
#to move a directory
import shutil

source_dir = '/content/drive/MyDrive/dataset/rust_dataset/og_data/no_rust/rusted'
destination_dir = '/content/drive/MyDrive/dataset/rust_dataset/og_data'

# Move the directory
shutil.move(source_dir, destination_dir)

print("Directory moved successfully.")

Directory moved successfully.


In [ ]:
#to move a directory
import shutil

source_dir = '/content/drive/MyDrive/dataset/rust_dataset/nonrusted'
destination_dir = '/content/drive/MyDrive/dataset/rust_dataset/og_data'

# Move the directory
shutil.move(source_dir, destination_dir)

print("Directory moved successfully.")

Directory moved successfully.


In [ ]:
import os
import shutil

# Define the directories
dataset_dir = '/content/drive/MyDrive/dataset/rust_dataset/data'  # Path to your dataset directory
labels = ['no_rust', 'rust']  # Define your labels

# Create label directories if they don't exist
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    os.makedirs(label_dir, exist_ok=True)

# Function to move files to corresponding label directories
def label_data(file, label):
    src = os.path.join(dataset_dir, label, file)
    dst = os.path.join(dataset_dir, label, file)
    shutil.move(src, dst)

# Iterate through the labels (folder names) in the dataset directory
for label in labels:
    # List all files in the label directory
    files = os.listdir(os.path.join(dataset_dir, label))
    # Move files to corresponding label directories
    for file in files:
        label_data(file, label)

print("Labeling completed.")


Labeling completed.


In [ ]:
import os
import random
import shutil

# Define the directories
dataset_dir = '/content/drive/MyDrive/dataset/rust_dataset/data'
train_dir = '/content/drive/MyDrive/dataset/rust_dataset/training'
val_dir = '/content/drive/MyDrive/dataset/rust_dataset/validation'
test_dir = '/content/drive/MyDrive/dataset/rust_dataset/testing'

# Define the labels (folder names)
labels = ['no_rust', 'rust']

# Define the ratios for splitting
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)
    for label in labels:
        os.makedirs(os.path.join(directory, label), exist_ok=True)

# Iterate through each label directory and split the images
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    images = os.listdir(label_dir)
    random.shuffle(images)

    num_images = len(images)
    train_split = int(train_ratio * num_images)
    val_split = int((train_ratio + val_ratio) * num_images)

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    for img in train_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(train_dir, label, img)
        shutil.move(src, dst)

    for img in val_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(val_dir, label, img)
        shutil.move(src, dst)

    for img in test_images:
        src = os.path.join(label_dir, img)
        dst = os.path.join(test_dir, label, img)
        shutil.move(src, dst)

print("Dataset split completed.")


Dataset split completed.


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = '/content/drive/MyDrive/dataset/rust_dataset/training'
val_dir = '/content/drive/MyDrive/dataset/rust_dataset/validation'

# Define parameters
batch_size = 32
img_height = 224
img_width = 224

# Create ImageDataGenerator for training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create ImageDataGenerator for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Get the list of classes in the training directory
classes = os.listdir(train_dir)

# Load and label training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and label validation data, excluding .ipynb_checkpoints
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes
)


Found 1454 images belonging to 2 classes.
Found 485 images belonging to 2 classes.


In [ ]:
#To know how many files are in the directory
import os

directory = '/content/drive/MyDrive/dataset/rust_dataset/testing'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)

Number of files in the directory: 486


In [ ]:
#To know how many files are in the directory
import os

directory = '/content/drive/MyDrive/dataset/rust_dataset/training'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)

Number of files in the directory: 1454


In [ ]:
#To know how many files are in the directory
import os

directory = '/content/drive/MyDrive/dataset/rust_dataset/validation'  # Change this to your directory path
file_count = sum(len(files) for _, _, files in os.walk(directory))

print("Number of files in the directory:", file_count)

Number of files in the directory: 485


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Initialize the model
model = Sequential()

# Add convolutional layers with increasing depth
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so use sigmoid activation

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Augment training data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/rust_dataset/training', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')

# Load validation data
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/dataset/rust_dataset/validation', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')

# Train the model
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=20, validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/rust_dataset/testing', target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')
eval_result = model.evaluate(test_generator)

print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])


Found 1454 images belonging to 2 classes.
Found 485 images belonging to 2 classes.
Epoch 1/20
45/45 [==============================] - 110s 2s/step - loss: 0.6778 - accuracy: 0.5999 - val_loss: 0.5935 - val_accuracy: 0.6333
Epoch 2/20
45/45 [==============================] - 108s 2s/step - loss: 0.5913 - accuracy: 0.6575 - val_loss: 0.5305 - val_accuracy: 0.6979
Epoch 3/20
45/45 [==============================] - 107s 2s/step - loss: 0.5185 - accuracy: 0.7328 - val_loss: 0.4816 - val_accuracy: 0.7708
Epoch 4/20
45/45 [==============================] - 109s 2s/step - loss: 0.4752 - accuracy: 0.7630 - val_loss: 0.4686 - val_accuracy: 0.7708
Epoch 5/20
45/45 [==============================] - 106s 2s/step - loss: 0.4341 - accuracy: 0.7813 - val_loss: 0.4202 - val_accuracy: 0.8000
Epoch 6/20
45/45 [==============================] - 101s 2s/step - loss: 0.3941 - accuracy: 0.8186 - val_loss: 0.3634 - val_accuracy: 0.8354
Epoch 7/20
45/45 [==============================] - 107s 2s/step - loss

In [ ]:
# Save the model
model.save('rust_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os

# Check if the model file exists
model_file = 'rust_model.h5'
if os.path.exists(model_file):
    print("Model saved successfully as", model_file)
else:
    print("Model saving failed")

Model saved successfully as rust_model.h5


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('/content/rust_model.h5')  # Replace 'your_model.h5' with the path to your trained model

# Function to preprocess input image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_image(image_path):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    return predictions

# Function to interpret predictions
def interpret_predictions(predictions):
    class_index = int(predictions[0][0])  # Assuming predictions is of shape (1, 1)
    confidence = predictions[0][0] if class_index == 1 else 1 - predictions[0][0]
    if class_index == 0:
        return "Not rusted ", confidence
    elif class_index == 1:
        return "Rusted pipe is found !", confidence

# Example usage
image_path = '/content/istockphoto-1062920970-2048x2048.jpg'
predictions = predict_image(image_path)
interpretation, confidence = interpret_predictions(predictions)
print(interpretation)
print(f"Confidence: {confidence}")


1/1 [==============================] - 0s 173ms/step
Rusted pipe is found !
Confidence: 1.0
